# Rotman Data Science Competition
### Section 6: Picking Top 1000 Products

## 0. Imports


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

## 1. Load Data


In [2]:
# Make dummy substitute data file
def make_dummy_substitute_data() -> bool:
    DATA_PATH = "data/mma_mart.csv"
    data = pd.read_csv(DATA_PATH)
    data["substitute"] = data["product_id"]
    data["frozen"] = [False] * data.shape[0]
    data["refrigerated"] = [False] * data.shape[0]
    return data
# data = make_dummy_substitute_data()

In [3]:
DATA_WTH_SUBS_PATH = "substitute_df.csv"
data = pd.read_csv(DATA_WTH_SUBS_PATH)
# data.head()
data.shape

(35070, 11)

## 2. Baseline Model

In [4]:
def baseline(data: pd.DataFrame, k: int, use_substitutes=True) -> pd.DataFrame:
    """ Return the top k items by sales

    Precondition:
    k is less than the number of items in the dataset
    """

    # Account for whether we are using substitutes or not
    if use_substitutes:
        column = "substitute"
    else:
        column = "product_id"

    # Count number of unique orders for each product or substitute
    sales = pd.DataFrame(data.groupby(column)["order_id"].nunique())

    # Rename column to sales and sort by sales
    sales.rename(columns = {"order_id" : "sales"}, inplace=True)
    sales.sort_values(by="sales", ascending=False, inplace=True)
    sales.reset_index(inplace=True)

    # Catch errors for debugging purposes
    if k > sales.shape[0]:
        raise ValueError("k must be less than the number of items in the dataset")

    # Return top k items by sales (i.e. top k items with most orders)
    return sales.iloc[:k][column].tolist()

## 3. Processing Data For Metric B Optimization


In [5]:
def add_met_b_column(data, use_substitutes=True) -> None:
    """ Add a column to the data that is the metric b impact score for each product calculated over the full dataset """
    order_id_to_order_size = data.groupby("order_id")["order_id"].count()
    mma_data_aug = data.copy()
    mma_data_aug["order_size"] = mma_data_aug["order_id"].map(order_id_to_order_size)

    mma_data_aug.loc[:, "portion_of_order"] = 1 / mma_data_aug["order_size"]

    if use_substitutes:
        column = "substitute"
    else:
        column = "product_id"

    id_to_met_b_score = mma_data_aug.groupby(column)["portion_of_order"].sum()
    data["met_b_score"] = data[column].map(id_to_met_b_score)
    return None

In [6]:
add_met_b_column(data)

In [7]:
data.sort_values(by="substitute", ascending=False).head(100)

,product_name,total_quantity_sold,order_id,product_id,aisle_id,aisle,department_id,department,substitute,frozen,refrigerated,met_b_score
18436,Cucumber,4,3735,6581,83,fresh vegetables,4,produce,49683,False,False,1.625000
8548,Cucumber Seedless,15,6037,34809,83,fresh vegetables,4,produce,49683,False,False,1.625000
15,Cucumber Kirby,2905,1,49683,83,fresh vegetables,4,produce,49683,False,False,1.625000
17699,Chicken and Cheese Taquitos,4,39357,11716,129,frozen appetizers sides,1,frozen,49678,True,False,2.325000
32365,Pierogi & Onion,1,61336,46551,38,frozen meals,1,frozen,49678,True,False,2.325000
...,...,...,...,...,...,...,...,...,...,...,...,...
13743,Mandarin Orange Sparkling Water,7,6460,5711,115,water seltzer sparkling water,7,beverages,49520,False,False,15.369553
12717,Tangerine Lemongrass Sparkling Water,8,14608,39199,115,water seltzer sparkling water,7,beverages,49520,False,False,15.369553
16009,Mandarin Orange Sparkling Seltzer Water,5,15567,11194,115,water seltzer sparkling water,7,beverages,49520,False,False,15.369553
24391,Black Raspberry Water,2,89519,33142,100,missing,21,missing,49520,False,False,15.369553


## 4. Optimization and Finding Top 1000 Products


In [8]:
# trim data
my_data = data.drop(columns=["aisle_id", "aisle", "department_id", "department"])
my_data.head()

,product_name,total_quantity_sold,order_id,product_id,substitute,frozen,refrigerated,met_b_score
0,Banana,14494,10,24852,24852,False,False,22.188600
1,Bag of Organic Bananas,11694,1,13176,13176,False,False,3.503968
2,Organic Strawberries,8081,10,21137,21137,False,False,17.255952
3,Organic Baby Spinach,7369,3,21903,21903,False,False,10.514652
4,Organic Hass Avocado,6411,1,47209,47209,False,False,4.249242


In [9]:
def max_metric_a_aisle(data: pd.DataFrame, k:int, use_substitutes=True) -> list:
    """ Return the aisle that maximizes metric a over the data. Do this by first getting the item with most sales, then getting the item with most sales in orders that do not contain the first item, and so on. For why this produces max list, see proof writeup.
    """
    return _max_metric_a_recur_helper(data, [], k, use_substitutes=use_substitutes)

def _max_metric_a_recur_helper(data: pd.DataFrame, aisle: list, k, use_substitutes=True) -> list:
    """ Return the aisle that maximizes metric a over the data. Do this by first getting the item with most sales, then getting the item with most sales in orders that do not contain the first item, and so on.

    Precondition:
    - data must be a subset of the test data
    - aisle must be a subset of the top 1000 items by sales
    """
    # Base case
    if len(aisle) == k:
        return aisle
    elif data.shape[0] == 0:
        return aisle
    else:
        # Get the top item by sales in data
        top_item = _get_top_product(data, use_substitutes=use_substitutes)

        # Add top item to aisle
        aisle.append(top_item)

        # Filter data to only include orders that do not contain the top item
        if use_substitutes:
            top_item_orders = data[data["substitute"] == top_item]
        else:
            top_item_orders = data[data["product_id"] == top_item]
        data_excluding_top_item = data[~data["order_id"].isin(top_item_orders["order_id"])]

        # Recursively call helper function
        return _max_metric_a_recur_helper(data_excluding_top_item, aisle, k, use_substitutes=use_substitutes)

def _get_top_product(data: pd.DataFrame, use_substitutes=True) -> list:
    """ Return the top item by sales in data. """

    if use_substitutes:
        column = "substitute"
    else:
        column = "product_id"

    sales = pd.DataFrame(data.groupby(column)["order_id"].nunique())

    # Rename column to sales
    sales.rename(columns = {"order_id": "sales"}, inplace=True)

    # Get top item by sales
    sales.reset_index(inplace=True) # Reset index so we don't get multiple items if there is a tie
    top_item =  sales.loc[sales["sales"].idxmax()][column]

    return top_item

In [10]:
def max_metric_b_aisle(data: pd.DataFrame, k: int, use_substitutes=True) -> list:
    """ Return the aisle that maximizes metric b over the data. Do this by picking the thousand items with the highest metric b score.
    """
    if data.shape[0] < k:
        raise ValueError("k must be less than the number of items in the dataset")

    if use_substitutes:
        column = "substitute"
    else:
        column = "product_id"

    data_copy = data.copy()
    add_met_b_column(data_copy)

    met_b_top_k = data_copy.sort_values(by="met_b_score", ascending=False)
    met_b_top_k.drop(columns="order_id", inplace=True)
    met_b_top_k.drop_duplicates(subset=column, inplace=True)
    met_b_top_k.reset_index(drop=True, inplace=True)
    met_b_top_k = met_b_top_k.iloc[:k][column].tolist()
    return met_b_top_k

In [40]:
def get_max_aisle(data: pd.DataFrame, k: int, use_substitutes=True) -> list:
    """ Make our aisle of k items """
    if data.shape[0] < k:
        raise ValueError("k must be less than the number of items in the dataset")

    return max_metric_b_aisle(data, k, use_substitutes=use_substitutes)

## 5. Combining Code

In [41]:
def make_1000_items(data: pd.DataFrame, use_substitutes=True) -> list:
    """ Make our aisle of 1000 items """
    data_copy = data.copy()
    add_met_b_column(data_copy)
    return get_max_aisle(data_copy, 1000, use_substitutes=use_substitutes)

## 6. Evaluation


### Defining Metrics

In [13]:
def metric_a(test_data: pd.DataFrame, insta_aisle: list, use_substitutes=True) -> float:
    """ Return the number of orders that utilize the in-aisle items as a percentage of the total number of orders in the test_data dataset.

    Precondition:
    len(test_data) > 0
    """
    # Account for whether we are using substitutes or not
    if use_substitutes:
        column = "substitute"
    else:
        column = "product_id"

    # Count the orders that utilize in-aisle items
    ## Filter the data to only include orders that purchased at least one item in the aisle
    purchases_in_aisle = test_data[test_data[column].isin(insta_aisle)]

    ## Count the number of orders that include at least one item in the aisle
    orders_in_aisle = purchases_in_aisle["order_id"].nunique()

    ## Count the total number of orders in the test data
    total_orders = test_data["order_id"].nunique()

    # Check for errors for debugging purposes
    if total_orders <= 0:
        raise ValueError("There must be at least one order in the test data")

    # Return the percentage of orders that utilize in-aisle items
    result = (orders_in_aisle / total_orders) * 100

    return result

In [14]:
def metric_b(test_data: pd.DataFrame, insta_aisle: list, use_substitutes=True) -> float:
    """ Calculate the percentage of items in each order that utilize in-aisle items and return the average of these percentages.
    """

    # Account for whether we are using substitutes or not
    if use_substitutes:
        column = "substitute"
    else:
        column = "product_id"

    # Copy data to prevent altering test_data
    test_data_copy = test_data.copy()

    # Add column that is 1 if the product id is in insta_aisle and 0 if not
    test_data_copy["is_utilized"] = test_data_copy[column].isin(insta_aisle).astype(int)

    # Sum the new column and divide by total number of items in order
    ## Get a map mapping order_id to the number of items in the order that are utilized
    order_to_ultilized_items = test_data_copy.groupby("order_id")["is_utilized"].sum()

    ## Get a map mapping order_id to the total number of items in the order
    order_to_total_items = test_data_copy.groupby("order_id")["order_id"].count()

    ## Divide the two maps to get order to the percentage of items in the order that are utilized
    order_to_precent_ultilization = order_to_ultilized_items / order_to_total_items


    # Return average of percentage ultilization over all orders
    return order_to_precent_ultilization.mean() * 100


In [15]:
def metric_average(test_data: pd.DataFrame, insta_aisle: list, use_substitutes=True) -> float:
    """ Return the average of metric a and metric b """
    return (metric_a(test_data, insta_aisle, use_substitutes) + metric_b(test_data, insta_aisle, use_substitutes)) / 2

In [16]:
def get_num_frozen_and_refrigerated(data, aisle, use_substitutes=True) -> tuple:
    """ Return the number of frozen and refrigerated items in the aisle """
    # Get the number of frozen and refrigerated items in the aisle
    if use_substitutes:
        column = "product_id" # Don't double count frozen and refrigerated items
    else:
        column = "product_id"
    data.drop_duplicates(subset=column, inplace=True)
    frozen = data[data[column].isin(aisle)]["frozen"].astype(int).sum()
    refrigerated = data[data[column].isin(aisle)]["refrigerated"].astype(int).sum()
    return frozen, refrigerated

### Get Training and Testing Sets

In [17]:
product_id_to_substitute = data.set_index("product_id")["substitute"]
product_id_to_frozen = data.set_index("product_id")["frozen"]
product_id_to_refrigerated = data.set_index("product_id")["refrigerated"]

In [18]:
mma_data = pd.read_csv("data/mma_mart.csv")
mma_data.head()

,order_id,product_id,product_name,aisle_id,aisle,department_id,department
0,1,49302,Bulgarian Yogurt,120,yogurt,16,dairy eggs
1,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,other creams cheeses,16,dairy eggs
2,1,10246,Organic Celery Hearts,83,fresh vegetables,4,produce
3,1,49683,Cucumber Kirby,83,fresh vegetables,4,produce
4,1,43633,Lightly Smoked Sardines in Olive Oil,95,canned meat seafood,15,canned goods


In [19]:
mma_data["substitute"] = mma_data["product_id"].map(product_id_to_substitute)
mma_data["frozen"] = mma_data["product_id"].map(product_id_to_frozen)
mma_data["refrigerated"] = mma_data["product_id"].map(product_id_to_refrigerated)
mma_data.head()

,order_id,product_id,product_name,aisle_id,aisle,department_id,department,substitute,frozen,refrigerated
0,1,49302,Bulgarian Yogurt,120,yogurt,16,dairy eggs,47912,False,True
1,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,other creams cheeses,16,dairy eggs,11109,False,True
2,1,10246,Organic Celery Hearts,83,fresh vegetables,4,produce,44359,False,False
3,1,49683,Cucumber Kirby,83,fresh vegetables,4,produce,49683,False,False
4,1,43633,Lightly Smoked Sardines in Olive Oil,95,canned meat seafood,15,canned goods,29195,False,False


In [20]:
TRAIN_TEST_SEED = 42 # Specify random state 42 for reproducibility.
TEST_SIZE = 0.2 # Use 20% of the data for testing.
orders = mma_data["order_id"].unique() # Get all unique order ids

# Split the orders into train and test sets
train_orders, test_orders = train_test_split(orders, test_size=TEST_SIZE, random_state=TRAIN_TEST_SEED)

# Retrieve all the purchases in the train and test order sets
train_data = mma_data[mma_data["order_id"].isin(train_orders)]
test_data = mma_data[mma_data["order_id"].isin(test_orders)]

In [21]:
train_data.shape

(790099, 10)

In [22]:
# Free up memory
data = 0

### Evaluate on the train test split

In [42]:
our_aisle = make_1000_items(train_data, use_substitutes=True)
our_aisle_no_sub = make_1000_items(train_data, use_substitutes=False)

In [24]:
baseline_aisle = baseline(train_data, 1000, use_substitutes=True)
baseline_aisle_no_sub = baseline(train_data, 1000, use_substitutes=False)

In [25]:
max_aisle_met_a = max_metric_a_aisle(test_data, 1000, use_substitutes=True)
max_aisle_met_b = max_metric_b_aisle(test_data, 1000, use_substitutes=True)

In [26]:
def show_results(aisle, test_data, name="Aisle"):
    print(name)
    print("Metric A: ", metric_a(test_data, aisle))
    print("Metric B: ", metric_b(test_data, aisle))
    print("Average: ", metric_average(test_data, aisle))
    print()

In [43]:
show_results(our_aisle, test_data, name="Our Aisle")
show_results(our_aisle_no_sub, test_data, name="Our Aisle (No Substitutes)")
show_results(baseline_aisle, test_data, name="Baseline Aisle")
show_results(baseline_aisle_no_sub, test_data, name="Baseline Aisle (No Substitutes)")
show_results(max_aisle_met_a, test_data, name="Theoretical Maximum Aisle Metric A")
show_results(max_aisle_met_b, test_data, name="Theoretical Maximum Aisle Metric B")

Our Aisle
Metric A:  99.36116931568458
Metric B:  89.57402192252175
Average:  94.46759561910316

Our Aisle (No Substitutes)
Metric A:  77.28318086574333
Metric B:  25.410390644325137
Average:  51.34678575503423

Baseline Aisle
Metric A:  99.23340317882149
Metric B:  89.461903099833
Average:  94.34765313932724

Baseline Aisle (No Substitutes)
Metric A:  97.39357080799304
Metric B:  73.65808556591533
Average:  85.52582818695419

Theoretical Maximum Aisle Metric A
Metric A:  99.73424643532478
Metric B:  84.68696210695022
Average:  92.2106042711375

Theoretical Maximum Aisle Metric B
Metric A:  99.42249706137886
Metric B:  89.76667637352155
Average:  94.5945867174502



### Evaluate on the full dataset

In [28]:
%%time
# Make Our Aisles
our_aisle = make_1000_items(mma_data, use_substitutes=True)
our_aisle_no_sub = make_1000_items(mma_data, use_substitutes=False)

CPU times: user 25.8 s, sys: 5.12 s, total: 31 s
Wall time: 31.1 s


In [29]:
%%time
# Make Baseline Aisles
baseline_aisle = baseline(mma_data, 1000, use_substitutes=True)
baseline_aisle_no_sub = baseline(mma_data, 1000, use_substitutes=False)

CPU times: user 571 ms, sys: 54.3 ms, total: 625 ms
Wall time: 625 ms


In [30]:
%%time
# Get the theoretical maximum aisles
max_aisle_met_a = max_metric_a_aisle(mma_data, 1000, use_substitutes=True)
max_aisle_met_b = max_metric_b_aisle(mma_data, 1000,  use_substitutes=True)

CPU times: user 6.82 s, sys: 884 ms, total: 7.7 s
Wall time: 7.72 s


In [31]:
mma_data.shape

(987259, 10)

In [32]:
# Evaluate our aisles
print("Our Aisle")
print("Metric A: ", metric_a(mma_data, our_aisle))
print("Metric B: ", metric_b(mma_data, our_aisle))
print("Average: ", metric_average(mma_data, our_aisle))

print("\nOur Aisle (No Substitutes)")
print("Metric A: ", metric_a(mma_data, our_aisle_no_sub, use_substitutes=False))
print("Metric B: ", metric_b(mma_data, our_aisle_no_sub, use_substitutes=False))
print("Average: ", metric_average(mma_data, our_aisle_no_sub, use_substitutes=False))

# Evaluate baseline aisles
print("\nBaseline Aisle")
print("Metric A: ", metric_a(mma_data, baseline_aisle))
print("Metric B: ", metric_b(mma_data, baseline_aisle))
print("Average: ", metric_average(mma_data, baseline_aisle))

print("\nBaseline Aisle (No Substitutes)")
print("Metric A: ", metric_a(mma_data, baseline_aisle, use_substitutes=False))
print("Metric B: ", metric_b(mma_data, baseline_aisle, use_substitutes=False))
print("Average: ", metric_average(mma_data, baseline_aisle, use_substitutes=False))

# Evaluate theoretical maximum aisles (no substitutes)
print("\nTheoretical Maximum Aisle (No Substitutes)")
print("Metric A: ", metric_a(mma_data, max_aisle_met_a, use_substitutes=False))

print("\nTheoretical Maximum Aisle Metric B (No Substitutes)")
print("Metric B: ", metric_b(mma_data, max_aisle_met_b, use_substitutes=False))

print("\nTheoretical Maximum Aisle Average (No Substitutes)")
print("Average: ", (metric_a(mma_data, max_aisle_met_a, use_substitutes=False) + metric_b(test_data, max_aisle_met_b, use_substitutes=False)) / 2)

# Evaluate theoretical maximum aisles (with substitutes)
print("\nTheoretical Maximum Aisle (With Substitutes)")
print("Metric A: ", metric_a(mma_data, max_aisle_met_a, use_substitutes=True))

print("\nTheoretical Maximum Aisle Metric B (With Substitutes)")
print("Metric B: ", metric_b(mma_data, max_aisle_met_b, use_substitutes=True))

print("\nTheoretical Maximum Aisle Average (Impossible to Achieve)")
print("Average: ", (metric_a(mma_data, max_aisle_met_a, use_substitutes=True) + metric_b(test_data, max_aisle_met_b, use_substitutes=True)) / 2)

Our Aisle
Metric A:  98.89812231046784
Metric B:  80.82759378580327
Average:  89.86285804813556

Our Aisle (No Substitutes)
Metric A:  90.97748203571392
Metric B:  42.59184630788048
Average:  66.7846641717972

Baseline Aisle
Metric A:  99.19352365766152
Metric B:  89.51572307420527
Average:  94.35462336593339

Baseline Aisle (No Substitutes)
Metric A:  87.88752261506853
Metric B:  37.43603597974471
Average:  62.661779297406625

Theoretical Maximum Aisle (No Substitutes)
Metric A:  87.6166528676418

Theoretical Maximum Aisle Metric B (No Substitutes)
Metric B:  37.620506930301836

Theoretical Maximum Aisle Average (No Substitutes)
Average:  62.65364463649513

Theoretical Maximum Aisle (With Substitutes)
Metric A:  99.72401950262181

Theoretical Maximum Aisle Metric B (With Substitutes)
Metric B:  89.76192913446106

Theoretical Maximum Aisle Average (Impossible to Achieve)
Average:  94.74534793807169


In [33]:
# Get the number of frozen and refrigerated items in our aisle
our_frozen, our_refrigerated = get_num_frozen_and_refrigerated(mma_data, our_aisle)
print("Our Aisle")
print("Frozen: ", our_frozen)
print("Refrigerated: ", our_refrigerated)

Our Aisle
Frozen:  48
Refrigerated:  154


In [34]:
# Get the number of frozen and refrigerated items in baseline aisle
baseline_frozen, baseline_refrigerated = get_num_frozen_and_refrigerated(mma_data, baseline_aisle)
print("Baseline Aisle")
print("Frozen: ", baseline_frozen)
print("Refrigerated: ", baseline_refrigerated)

Baseline Aisle
Frozen:  90
Refrigerated:  248


In [35]:
len(set(our_aisle).intersection(set(baseline_aisle)))


563